In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import os
from statsmodels.tsa.statespace.sarimax import SARIMAX

E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
#Population values
population = {
    "Chiapas_2016-2017.csv": 5217908,
    "Colima_2016-2017.csv": 711235,
    "Guerrero_2016-2017.csv": 3533251,
    "Hidalgo_2016-2017.csv": 2858359,
    "NuevoLeon_2016-2017.csv": 5119504,
    "Oaxaca_2016-2017.csv": 3967889,
    "QuintanaRoo_2016-2017.csv": 1501562,
    "Tabasco_2016-2017.csv" : 2395272,
    "Veracruz_2016-2017.csv" : 8112505,
    "Yucatan_2016-2017.csv" : 2097175,
    
    "casanare_2016-2017.csv" : 356438,
    "cordoba_2016-2017.csv" : 1709603,
    "cundinamarca_2016-2017.csv" : 2680041,
    "huila_2016-2017.csv" : 1154804,
    "meta_2016-2017.csv" : 961292,
    "santander_2016-2017.csv" : 2061095,
    "santander_norte_2016-2017.csv" : 1355723,
    "tolima_2016-2017.csv" : 1408274,
    "valle_cauca_2016-2017.csv" : 4613377,
    
    "Alagoas_2016-2017.csv": 3375823,
    "Bahia_2016-2017.csv": 15344447,
    "Ceara_2016-2017.csv": 9020460,
    "Goias_2016-2017.csv": 6778772,
    "Maranhao_2016-2017.csv": 7000229,
    "MatoGrosso_2016-2017.csv": 3344544,
    "MinasGerais_2016-2017.csv": 21119536,
    "Para_2016-2017.csv": 8366628,
    "RioDeJaneiro_2016-2017.csv": 16718956,
    "SaoPaulo_2016-2017.csv": 45094866,
}

In [3]:
def formatFilename(filename):
    return filename.replace(".csv", "")

In [ ]:
test_index = 52 #Use first year to train, second to test
for (p,d,q) in [(p,d,q) for p in range(5) for d in range(2) for q in range(5)]:
    for (sp, sd, sq, seasonal) in [(sp, sd, sq, seasonal) for sp in range(5)\
                                   for sd in range(2) for sq in range(5) for seasonal in [0,1, 52]]:
        
        try:
            for country in ["Mexico", "Brazil", "Colombia"]:
                print(country)
                print(p,d,q)
                print(sp,sd,sq,seasonal)
                print("---")
                folder = "../../data/{}/processed_data".format(country)
                files = os.listdir(folder)
                

                for file in files:
                    
                    outFolder = "{},{},{}-{},{},{},{}".format(p,d,q, sp, sd, sq, seasonal)
                    if(not os.path.exists("{}/{}/{}".format(country, outFolder, file))):
                        print("Performing experiment in {}/{}/{}".format(country, outFolder, file))
                        dataset = pd.read_csv("{}/{}".format(folder, file))
                        formattedFilename = formatFilename(file)
                        #Preprocess data
                        dataset["Searches"] /= 100
                        state = file
                        dataset["Cases"] *= (100000/population[state]) # Transform to incidence per 100,000 habitants    

                        #Develop Sarimax model
                        model= SARIMAX(dataset["Cases"].values,
                                    exog=dataset["Searches"].values,
                                    trend='n', order=(p,d,q), seasonal_order=(sp,sd,sq,seasonal))
                        #Fit model
                        results = model.fit()
                    #     print(results.summary())

                        #Predict for next year
                        predictions = results.predict(
                        start = test_index,             
    #                     dynamic= False, #lagged values are used in next prediction
    # #                     exog=dataset["Searches"][test_index:].values.reshape(52,1)
                        )

                        #Store results
                        predictions_df = pd.DataFrame()
                        predictions_df["predictions_incidence"] = predictions
                        predictions_df["observed_incidence"] = dataset["Cases"][test_index:].values

                        predictions_df["Predicted Cases"] = predictions * (population[state] / 100000)
                        predictions_df["Observed Cases"] = dataset["Cases"][test_index:].values * (population[state] / 100000)

                        predictions_df["error"] = predictions_df["Observed Cases"] - predictions_df["Predicted Cases"]
                        predictions_df.set_index(dataset[52:]["Date"], inplace=True)

                        outFolder = "{},{},{}-{},{},{},{}".format(p,d,q, sp, sd, sq, seasonal)

                        if(not os.path.isdir("{}/{}".format(country, outFolder))):
                            os.mkdir("{}/{}".format(country, outFolder))

                        if(not os.path.isdir("{}/{}/{}".format(country, outFolder,file))):
                            os.mkdir("{}/{}/{}".format(country, outFolder, file))

                        predictions_df.to_csv("{}/{}/{}/Arimax-{}.csv".format(country, outFolder, file, formattedFilename))

                        #Save images
                        plt.clf()

                        cols = ['Observed Cases', 'Predicted Cases']
                        colors = ['#2962FF', '#FF9800']

                        predictions_df[cols].plot(
                            figsize=(10, 10),
                            grid=True,
                            color=colors
                        )

                        plt.title("Arimax Model\n{}".format(formattedFilename.replace("_2016-"," ")))
                        ax = plt.gca()
                        ax.set_facecolor((0.9, 0.9, 0.9, 0.7))
                        plt.xlabel("Date")
                        plt.ylabel("ZIKV Cases")
                        plt.grid(linestyle='dashed', linewidth=1.5)
                        fig = plt.gcf()
                        fig.savefig("{}/{}/{}/Arimax-{}.png".format(country, outFolder, file, formattedFilename))
                        plt.close("all")
                    else:
                        print("Experiment already exists")
        except Exception as e:
            print(e)

Mexico
0 0 0
0 0 0 0
---
Performing experiment in Mexico/0,0,0-0,0,0,0/Chiapas_2016-2017.csv
Number of states in statespace model must be a positive number.
Mexico
0 0 0
0 0 0 1
---
Performing experiment in Mexico/0,0,0-0,0,0,1/Chiapas_2016-2017.csv
Number of states in statespace model must be a positive number.
Mexico
0 0 0
0 0 0 52
---
Performing experiment in Mexico/0,0,0-0,0,0,52/Chiapas_2016-2017.csv
Number of states in statespace model must be a positive number.
Mexico
0 0 0
0 0 1 0
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Brazil
0 0 0
0 0 1 0
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Colombia
0 0 0
0 0 1 0
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Mexico
0 0 0
0 0 1 1
---
Experiment already exists
Experimen

E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\tsa\statespace\tools.py:405: RuntimeWarning: invalid value encountered in sqrt
  x = r / ((1 - r**2)**0.5)


Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Colombia
0 0 0
1 0 2 0
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Mexico
0 0 0
1 0 2 1
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Brazil
0 0 0
1 0 2 1
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Colombia
0 0 0
1 0 2 1
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Mexico
0 0 0
1 0 2 52
---
Performing experiment in Mexico/0,0,0-1,0,2,52/Chiapas_2016-2017.csv
maxlag should be < nobs
Mexico
0 0 0
1 0 3 0
---
Experiment already exists
Experiment already exists
Performing experiment in Mexico/0,0,0-1,0,3,0/NuevoLeon_2016-2017.csv
Non-st

2 1 3 1
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Brazil
0 0 0
2 1 3 1
---
Experiment already exists
Performing experiment in Brazil/0,0,0-2,1,3,1/MatoGrosso_2016-2017.csv
array must not contain infs or NaNs
Mexico
0 0 0
2 1 3 52
---
Performing experiment in Mexico/0,0,0-2,1,3,52/Chiapas_2016-2017.csv
maxlag should be < nobs
Mexico
0 0 0
2 1 4 0
---
Performing experiment in Mexico/0,0,0-2,1,4,0/Chiapas_2016-2017.csv
Invalid value for design matrix. Requires a 2- or 3-dimensional array, got 1 dimensions
Mexico
0 0 0
2 1 4 1
---
Experiment already exists
Performing experiment in Mexico/0,0,0-2,1,4,1/Guerrero_2016-2017.csv
array must not contain infs or NaNs
Mexico
0 0 0
2 1 4 52
---
Performing experiment in Mexico/0,0,0-2,1,4,52/Chiapas_2016-2017.csv
maxlag should be < nobs
Mexico
0 0 0
3 0 0 0
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exi

4 0 3 52
---
Performing experiment in Mexico/0,0,0-4,0,3,52/Chiapas_2016-2017.csv
maxlag should be < nobs
Mexico
0 0 0
4 0 4 0
---
Performing experiment in Mexico/0,0,0-4,0,4,0/Chiapas_2016-2017.csv
array must not contain infs or NaNs
Mexico
0 0 0
4 0 4 1
---
Performing experiment in Mexico/0,0,0-4,0,4,1/Chiapas_2016-2017.csv
array must not contain infs or NaNs
Mexico
0 0 0
4 0 4 52
---
Performing experiment in Mexico/0,0,0-4,0,4,52/Chiapas_2016-2017.csv
maxlag should be < nobs
Mexico
0 0 0
4 1 0 0
---
Performing experiment in Mexico/0,0,0-4,1,0,0/Chiapas_2016-2017.csv
Invalid value for design matrix. Requires a 2- or 3-dimensional array, got 1 dimensions
Mexico
0 0 0
4 1 0 1
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Brazil
0 0 0
4 1 0 1
---
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Experiment already exists
Colombia
0 0 0
4 1 0 1
---

...

Performing experiment in Colombia/3,0,0-4,1,1,1/santander_2016-2017.csv
Performing experiment in Colombia/3,0,0-4,1,1,1/santander_norte_2016-2017.csv


E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Performing experiment in Colombia/3,0,0-4,1,1,1/tolima_2016-2017.csv


E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Performing experiment in Colombia/3,0,0-4,1,1,1/valle_cauca_2016-2017.csv
Mexico
3 0 0
4 1 1 52
---
Performing experiment in Mexico/3,0,0-4,1,1,52/Chiapas_2016-2017.csv
maxlag should be < nobs
Mexico
3 0 0
4 1 2 0
---
Performing experiment in Mexico/3,0,0-4,1,2,0/Chiapas_2016-2017.csv
Invalid value for design matrix. Requires a 2- or 3-dimensional array, got 1 dimensions
Mexico
3 0 0
4 1 2 1
---
Performing experiment in Mexico/3,0,0-4,1,2,1/Chiapas_2016-2017.csv
Non-stationary starting autoregressive parameters found with `enforce_stationarity` set to True.
Mexico
3 0 0
4 1 2 52
---
Performing experiment in Mexico/3,0,0-4,1,2,52/Chiapas_2016-2017.csv
maxlag should be < nobs
Mexico
3 0 0
4 1 3 0
---
Performing experiment in Mexico/3,0,0-4,1,3,0/Chiapas_2016-2017.csv
Invalid value for design matrix. Requires a 2- or 3-dimensional array, got 1 dimensions
Mexico
3 0 0
4 1 3 1
---
Performing experiment in Mexico/3,0,0-4,1,3,1/Chiapas_2016-2017.csv
Non-stationary starting autoregressive para